In [9]:
import torch
import pprint
import vel
from vel.rl.models.stochastic_policy_model_separate import StochasticPolicyModelSeparateFactory
from vel.modules.input.normalize_observations import NormalizeObservationsFactory
from vel.rl.models.backbone.mlp import MLPFactory
from vel.util.random import set_seed
from vel.rl.env.mujoco import MujocoEnv

In [10]:
state_dict = torch.load('/Users/sgillen/work_dir/ucsb/sgillen_vel/output/checkpoints/walker_ppo/7/checkpoint_00000489.data' ,map_location = 'cpu')
hidden_dict =  torch.load('/Users/sgillen/work_dir/ucsb/sgillen_vel/output/checkpoints/walker_ppo/7/checkpoint_hidden_00000489.data', map_location = 'cpu')

In [11]:
seed = 1002
set_seed(seed) # Set random seed in python std lib, numpy and pytorch
env = MujocoEnv('Walker2d-v2').instantiate(seed=seed)


policy_in_size = state_dict['policy_backbone.model.0.weight'].shape[1]
value_in_size = state_dict['value_backbone.model.0.weight'].shape[1]


model_factory = StochasticPolicyModelSeparateFactory(
    input_block=NormalizeObservationsFactory(input_shape=17),
    policy_backbone=MLPFactory(input_length=policy_in_size, hidden_layers=[64, 4], activation='tanh'),
    value_backbone=MLPFactory(input_length=value_in_size, hidden_layers=[64, 4], activation='tanh'),
)

#sgillen - pretty sure this takes care of the output sizes for us automatically
model = model_factory.instantiate(action_space=env.action_space)
model.load_state_dict(state_dict)

env.allow_early_resets = True

In [12]:
ob = env.reset()    
rewards = []

while True:
    #action = model.step(torch.Tensor(ob)).detach().numpy()
    action = model.step(torch.Tensor(ob).view(1,-1))['actions'].detach().numpy()
    ob, reward , done, _ =  env.step(action)
    
    rewards.append(reward)
    
    #if reward == 1:
    #    print("balanced")
    env.render()
    if done:
        print(max(rewards))
        rewards
        ob = env.reset()

Creating window glfw
6.1596832664351515
6.1596832664351515
6.1596832664351515
6.1596832664351515
6.508740970721516
6.508740970721516
6.508740970721516
6.508740970721516
6.508740970721516
6.508740970721516
6.521505105964191
6.521505105964191
6.521505105964191


KeyboardInterrupt: 

In [28]:
state_dict.keys()

odict_keys(['input_block.running_mean', 'input_block.running_var', 'input_block.count', 'policy_backbone.model.0.weight', 'policy_backbone.model.0.bias', 'policy_backbone.model.2.weight', 'policy_backbone.model.2.bias', 'value_backbone.model.0.weight', 'value_backbone.model.0.bias', 'value_backbone.model.2.weight', 'value_backbone.model.2.bias', 'action_head.head.log_std', 'action_head.head.linear_layer.weight', 'action_head.head.linear_layer.bias', 'value_head.linear_layer.weight', 'value_head.linear_layer.bias'])

In [ ]:
state_dict['policy_backbone.model.0.weight'].shape